In [1]:
import torch
import pandas as pd
import numpy as np
import math

import textwrap

from diffusers import StableDiffusionPipeline
from huggingface_hub import notebook_login

import matplotlib.pyplot as palt
from PIL import Image

notebook_login()

2025-12-02 21:40:57.314150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764711657.503949      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764711657.557481      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [41]:
pipe = StableDiffusionPipeline.from_pretrained(
    "Manojb/stable-diffusion-2-1-base", # "CompVis/stable-diffusion-v1-4",
    safety_checker=None,
    torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [42]:
def generate(prompts: list[str], num_inference_steps: int=30, guidance_scale: int=7.5):
  images = pipe(prompts, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images

  return images


def get_text_encoder_activations(pipe: StableDiffusionPipeline, prompts: list[str]):
    inputs = pipe.tokenizer(
        prompts,
        padding="max_length",
        max_length=pipe.tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt"
    ).to(pipe.text_encoder.device)

    with torch.inference_mode():
        outputs = pipe.text_encoder(**inputs, output_hidden_states=True)

    return outputs.hidden_states



def get_unet_residual_stream(
    pipe,
    prompts: list[str],
    num_inference_steps: int = 30,
    guidance_scale: float = 7.5
):
    residuals_dict = {}
    handles = []

    def save_residuals(name):
        def hook(module, input, output):
            # I ResnetBlock restituiscono una tupla: (hidden_state, residual)
            residual = output[1] if isinstance(output, tuple) else output
            # Accumula i residuals in una lista
            if isinstance(residual, torch.Tensor):
                residuals_dict.setdefault(name, []).append(residual.detach().cpu())
            else:
                residuals_dict.setdefault(name, []).extend([r.detach().cpu() for r in residual])
        return hook

    for i, block in enumerate(pipe.unet.down_blocks):
        for j, resnet in enumerate(block.resnets):
            handles.append(resnet.register_forward_hook(save_residuals(f"down_block_{i}_resnet_{j}")))

    for j, resnet in enumerate(pipe.unet.mid_block.resnets):
        handles.append(resnet.register_forward_hook(save_residuals(f"mid_block_resnet_{j}")))

    for i, block in enumerate(pipe.unet.up_blocks):
        for j, resnet in enumerate(block.resnets):
            handles.append(resnet.register_forward_hook(save_residuals(f"up_block_{i}_resnet_{j}")))

    with torch.inference_mode():
        _ = pipe(
            prompts,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
        )

    # Rimuovi gli hook
    for h in handles:
        h.remove()

    return residuals_dict

def show_images(images: list[Image.Image], prompts: list[str], cols: int = 2, width: int = 40) -> None:
    assert len(images) == len(prompts)

    rows = math.ceil(len(images) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(3 * cols, 3 * rows))

    if isinstance(axes, np.ndarray):
        axes = axes.flatten()
    else:
        axes = [axes]

    for ax in axes[len(images):]:
        ax.axis('off')

    for ax, img, prompt in zip(axes, images, prompts):
        ax.imshow(img)
        ax.axis('off')
        wrapped_prompt = "\n".join(textwrap.wrap(prompt, width=width))
        ax.text(0.5, -0.05, wrapped_prompt, fontsize=10, ha='center', va='top', transform=ax.transAxes)
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_color('black')
            spine.set_linewidth(1)

    plt.tight_layout()
    plt.show()

In [4]:
dogs_dataset = pd.read_csv('/kaggle/input/prompts-steering/dogs.csv')

dog_prompts = dogs_dataset['positive'].tolist()
non_dog_prompts = dogs_dataset['negative'].tolist()

In [ ]:
batch_size = 3
images_no = 3 # len(dog_prompts)

all_images = []
all_prompts = []

for i in range(0, images_no, batch_size):
    pos_batch = dog_prompts[i:i+batch_size]
    neg_batch = non_dog_prompts[i:i+batch_size]

    pos_images = generate(pos_batch)
    neg_images = generate(neg_batch)

    for p_img, n_img, p_prompt, n_prompt in zip(pos_images, neg_images, pos_batch, neg_batch):
        all_images.append(p_img)
        all_images.append(n_img)
        all_prompts.append(p_prompt)
        all_prompts.append(n_prompt)

show_images(all_images, all_prompts, cols=2)

In [47]:
dog_activations = get_unet_residual_stream(pipe, dog_prompts[:3])

  0%|          | 0/30 [00:00<?, ?it/s]

In [46]:
#print(pipe.unet)

for layer_name, activation in dog_activations.items():
    print(f'Layer {layer_name}: {len(activation)}')

Layer down_block_0_resnet_0: 4
Layer down_block_0_resnet_1: 4
Layer down_block_1_resnet_0: 4
Layer down_block_1_resnet_1: 4
Layer down_block_2_resnet_0: 4
Layer down_block_2_resnet_1: 4
Layer down_block_3_resnet_0: 4
Layer down_block_3_resnet_1: 4
Layer mid_block_resnet_0: 4
Layer mid_block_resnet_1: 4
Layer up_block_0_resnet_0: 4
Layer up_block_0_resnet_1: 4
Layer up_block_0_resnet_2: 4
Layer up_block_1_resnet_0: 4
Layer up_block_1_resnet_1: 4
Layer up_block_1_resnet_2: 4
Layer up_block_2_resnet_0: 4
Layer up_block_2_resnet_1: 4
Layer up_block_2_resnet_2: 4
Layer up_block_3_resnet_0: 4
Layer up_block_3_resnet_1: 4
Layer up_block_3_resnet_2: 4
